In [28]:
!pip install apyori

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Nel csv non è presente un header per il nome delle colonne, per cui nell'importazione del csv è necessario indicare header=None

In [30]:
dataset = pd.read_csv("Market_Basket_Optimisation.csv", header=None)

La libreria "apyori" non utilizza i pandas dataframe ma delle matrici formate da liste. E' necessario fare un pre processing dei dati per poterli avere nel formato corretto.

Il dataframe viene convertito in una matrice che ha come righe le transazioni e come colonne i prodotti acquistati nella transazione.

Inoltre, tutti i valori contenuti nelle liste devono essere stringhe.

In [31]:
transactions = []
for i in range(0, dataset.shape[0]):
  transactions.append(
    [str(dataset.values[i,j]) for j in range(0, dataset.shape[1])]
  )

min_support = permette di tener conto solo delle regole che hanno un effettiva rilevanza. Il numero va studiato in funzione della situazione.

In questo caso, si vogliono tener conto solo dei prodotti che vengono comprati almeno 3 volte al giorno.
I dati della tabella riguardano però un intera settimana, per cui in una settimana voglio vedere i prodotti comprati almeno 3*7=21 volte.

Il supporto è dato dal numero di osservazioni dell'elemento sul numero di osservazioni totali, per cui 21/7500.

min_confidence = indica in che percentuale l'elemento 2 è presente quando è presente anche l'elemento 1 (1 -> 2). Non esiste una regola esatta. Si può partire da un valore di default (0.8) e scendere di valore se le regole trovate fossero troppo poche o se i risultati non fossero soddisfacenti.

min_lift = indica la qualità/rilevanza di una regola. Il valore da utilizzare è basato principalmente sull'esperienza, in genere almeno 3.

min e max length = indica la lunghezza minima e massima delle regole. Se min e max sono uguali, vincolo ad avere una regola esattamente di quella lunghezza.

Con una lunghezza di 2, trovo solo regole del tipo "dato elemento 1 ho anche elemento 2"

In [32]:
from apyori import apriori
rules = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)

In [33]:
results = list(rules)
results

[RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'fromage blanc', 'honey'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

In [34]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


In [35]:
resultsinDataFrame.nlargest(n=10, columns="Lift")

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710
